In [1]:
# Assume a collection of playlists, P (which they refer to as the Case Base).

# User inputs to the algorithm:
# - seed song (s)
# - desired length (N)
# Hyperparameter (k)

import pandas as pd
import os, sys, time, random
import json
import numpy as np
import keras
import sqlite3
import math
import functools
import operator
import asyncio

k = 20
# seed = 'spotify:track:6I9VzXrHxO9rA9A5euc8Ak' # Britney Spears - Toxic
# seed = 'spotify:track:4aVuWgvD0X63hcOCnZtNFA' # Toto - Hold the Line
# seed = 'spotify:track:5pyw9ufwba90TEl8HMzdDI' # Ken Ashcorp - On The Rocks
# seed = 'spotify:track:0kTZWRzfmqDN4boykkjUXH'
seed = 'spotify:track:3mScGCzxiXA9OaHdBeuk7O'
conn = sqlite3.connect("/mnt/z/other/spotify_backup.db", check_same_thread=False)
cur = conn.cursor()

2023-04-04 14:30:28.598199: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-04 14:30:28.690790: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-04 14:30:28.693357: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-04 14:30:28.693369: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

In [2]:
def var_attr(l, gamma, delta):
    mul_attr = 1
    i = 0
    while i < len(l) - 1:
        j = i + 1
        while j < len(l) and j - i < gamma:
            if l[j] != 0 and l[i] != 0 and abs(l[j] - l[i]) > delta:
                mul_attr *= ((j-i) / gamma)
                i = j
                break
            j += 1

        i += 1

    return mul_attr

In [3]:
def var(p): # todo: fine-tuning deltas and gammas
    con = sqlite3.connect("/mnt/z/other/spotify_backup.db", check_same_thread=False)
    c = con.cursor()
    delta_diffs = [0.25, 0.25, 0.25, 0.25, 0.25]
    gammas = [3, 3, 3, 3, 3]
    feature_list = np.array(c.execute(f"SELECT tf.acousticness, tf.instrumentalness, tf.speechiness, tf.energy, tf.valence FROM track_features as tf JOIN playlist_tracks as pt WHERE tf.track_uri = pt.track_uri AND pt.pid = '{p}' ORDER BY pt.pindex ASC;").fetchall()).transpose()
    total_attrs = [var_attr(feature_list[i], gammas[i], delta_diffs[i]) for i in range(len(gammas))]
    total = 1
    for attr in total_attrs:
        total *= attr

    return total

In [4]:
def rel(q, t, checked=True):
    con = sqlite3.connect("/mnt/z/other/spotify_backup.db", check_same_thread=False)
    c = con.cursor()
    if not checked:
        if t in q:
            phi = int(c.execute(f"SELECT COUNT(*) FROM seq2_simple WHERE track_uri1 = '{q[0]}' AND track_uri2 = '{q[1]}';").fetchone()[0])
        else:
            return -1 # t not in q error
    else:
        phi = int(c.execute(f"SELECT COUNT(*) FROM seq2_simple WHERE track_uri1 = '{q[0]}' AND track_uri2 = '{q[1]}';").fetchone()[0])
    
    if False: # future variability in pattern size, adding weighting based on length
        alpha = 0.5
        theta = 2
        lq = len(q)
    
    beta = 0.5 # popularity weighting
    psi = 0
    x = q[0]
    if q[0] == t:
        x = q[1]

    psi = pow(int(c.execute(f"SELECT COUNT(*) FROM playlist_tracks WHERE track_uri = '{x}';").fetchone()[0]), beta)
    rel = phi / psi

    return rel

In [5]:
from multiprocessing import Pool
from multiprocessing import Process
def test(x):
    con = sqlite3.connect("/mnt/z/other/spotify_backup.db", check_same_thread=False)
    c = con.cursor()
    return c.execute(f"SELECT * FROM playlist_tracks LIMIT {x}, 100;").fetchall()[0]

with Pool(5) as p:
        print(p.map(test, [0, 100, 200, 300]))


[(0, 0, 'spotify:track:0UaMYEvWZi0ZqiDOoHU3YI'), (2, 9, 'spotify:track:7tIJDktakabGoHjwTTa35W'), (3, 45, 'spotify:track:0tOyrixMQ17NUznPIxYtVD'), (5, 2, 'spotify:track:1V4jC0vJ5525lEF1bFgPX2')]


In [6]:
containing = cur.execute(f"SELECT pid, pindex FROM playlist_tracks WHERE track_uri = '{seed}';").fetchall()
totals = []

prev_cases = [f"(pid = {c[0]} AND pindex-1 = {c[1]})" for c in containing]
prev_split = []
for x in range(0, len(prev_cases), 900):
    prev_split.append("OR".join(prev_cases[x:x+900]))

after_cases = [f"(pid = {c[0]} AND pindex+1 = {c[1]})" for c in containing]
after_split = []
for x in range(0, len(after_cases), 900):
    after_split.append("OR".join(after_cases[x:x+900]))

def fetch(x):
    con = sqlite3.connect("/mnt/z/other/spotify_backup.db", check_same_thread=False)
    c = con.cursor()
    return c.execute(f"SELECT pid, track_uri FROM playlist_tracks WHERE {x} ORDER BY pid;").fetchall()

In [7]:
print(len(prev_split))
print(len(after_split))

1
1


In [8]:
fetched_prev_tracks = []
fetched_after_tracks = []
# with Pool() as p:
# with Pool((max(len(prev_cases), len(after_cases)) // 900)*2) as p:
with Pool(64) as p:
    fetched_prev_tracks = p.map_async(fetch, prev_split)
    fetched_after_tracks = p.map_async(fetch, after_split)

    p.close()
    p.join()

# fetched_after_tracks.wait()
# fetched_prev_tracks.wait()


In [9]:
fetched_prev_tracks = [item for sublist in fetched_prev_tracks.get() for item in sublist]
fetched_after_tracks = [item for sublist in fetched_after_tracks.get() for item in sublist]
# f = open("fetched_lists.txt","w")
# f.write(str(fetched_prev_tracks))
# f.close()

# fetched_prev_tracks = dict(fetched_prev_tracks)
# fetched_after_tracks = dict(fetched_after_tracks)

# f = open("fetched_dicts.txt", "w")
# f.write(str(fpt))
# f.close()

# new_fpt = []
# for i in range(len(fetched_prev_tracks)):
#     j = i + 1
#     if j == len


In [10]:
fpt = np.transpose(fetched_prev_tracks)
fat = np.transpose(fetched_after_tracks)

In [11]:
def batch_phi(pre, songs, t):
    con = sqlite3.connect("/mnt/z/other/spotify_backup.db", check_same_thread=False)
    c = con.cursor()
    Q_s = "(" + ",".join(["'"+i+"'" for i in songs]) + ")" # song uris
    if not pre:
        phis = dict([[b[0], float(b[1])] for b in c.execute(f"SELECT track_uri2, COUNT(*) FROM seq2_simple WHERE track_uri1 = '{t}' AND track_uri2 in {Q_s} GROUP BY track_uri2 ORDER BY track_uri2;").fetchall()])
    else:
        phis = dict([[a[0], float(a[1])] for a in c.execute(f"SELECT track_uri1, COUNT(*) FROM seq2_simple WHERE track_uri1 in {Q_s} AND track_uri2 = '{t}' GROUP BY track_uri1 ORDER BY track_uri1;").fetchall()])

    return phis

In [12]:
def batch_rel(setting, t=seed):
    con = sqlite3.connect("/mnt/z/other/spotify_backup.db", check_same_thread=False)
    c = con.cursor()
    if setting == 0:
        print("got to here")
        Q = fpt
        print("LEN Q[0]:", len(Q[0]))
        Q_s = "(" + ",".join(["'"+i+"'" for i in Q[1]]) + ")" # song uris
            
        phis = dict([[b[0], float(b[1])] for b in c.execute(f"SELECT track_uri2, COUNT(*) FROM seq2_simple WHERE track_uri1 = '{t}' AND track_uri2 in {Q_s} GROUP BY track_uri2 ORDER BY track_uri2;").fetchall()])

    elif setting == 1:
        print("got to here alt")
        Q = fat
        Q_s = "(" + ",".join(["'"+i+"'" for i in Q[1]]) + ")"
        phis = dict([[a[0], float(a[1])] for a in c.execute(f"SELECT track_uri1, COUNT(*) FROM seq2_simple WHERE track_uri1 in {Q_s} AND track_uri2 = '{t}' GROUP BY track_uri1 ORDER BY track_uri1;").fetchall()])

    print("got to here 1")
    if False: # future variability in pattern size, adding weighting based on length
        alpha = 0.5
        theta = 2
        lq = len(q)
    
    beta = 0.5 # popularity weighting

    print("LEN PHI:", len(phis.keys()))
    Q_s_filtered = "(" + ",".join(["'"+i+"'" for i in phis.keys()]) + ")"
    counts = np.transpose(c.execute(f"SELECT COUNT(*) FROM playlist_tracks WHERE track_uri in {Q_s_filtered} GROUP BY track_uri ORDER BY track_uri;").fetchall())[0]
    psis = np.power(counts, beta)
    rels = np.divide(psis, list(phis.values()))

    print("got to here 2")
    print("rels:", str(rels))

    rel_d = {}
    pkeys = list(phis.keys())
    for z in range(len(Q[0])):
        if Q[1][z] in phis.keys():
            if Q[0][z] in rel_d.keys():
                rel_d[Q[0][z]] += rels[pkeys.index(Q[1][z])]
            else:
                rel_d[Q[0][z]] = rels[pkeys.index(Q[1][z])]
    
    print("rel_d:", rel_d)

    return rel_d

In [13]:
# def fetch_occurences(l, prev=0, t=seed):
#     con = sqlite3.connect("/mnt/z/other/spotify_backup.db", check_same_thread=False)
#     c = con.cursor()
#     if prev:
#         return np.transpose([[b[0], float(b[1])] for b in c.execute(f"SELECT track_uri2, COUNT(*) FROM seq2_simple WHERE track_uri1 = '{t}' AND track_uri2 in {Q_s} GROUP BY track_uri2 ORDER BY track_uri2;").fetchall()])
#     else:
#         return np.transpose([[a[0], float(a[1])] for a in c.execute(f"SELECT track_uri1, COUNT(*) FROM seq2_simple WHERE track_uri1 in {Q_s} AND track_uri2 = '{t}' GROUP BY track_uri1 ORDER BY track_uri1;").fetchall()])

In [14]:
# def batch_rel(setting, t=seed):
#     con = sqlite3.connect("/mnt/z/other/spotify_backup.db", check_same_thread=False)
#     c = con.cursor()
#     if setting == 0:
#         print("got to here")
#         Q = fetched_prev_tracks.keys()
#         print("LENQ", len(Q))
#         Q_s = "(" + ",".join(["'"+fetched_prev_tracks[i]+"'" for i in Q]) + ")"
        
#         # ps = []
#         # for x in range(0, len(fetched_prev_tracks), 900):
#         #     ps.append(",".join(fetched_prev_tracks[x:x+900]))
            
#         phis = dict([[b[0], float(b[1])] for b in c.execute(f"SELECT track_uri2, COUNT(*) FROM seq2_simple WHERE track_uri1 = '{t}' AND track_uri2 in {Q_s} GROUP BY track_uri2 ORDER BY track_uri2;").fetchall()])

#     elif setting == 1:
#         print("got to here alt")
#         Q = fetched_after_tracks.keys()
#         Q_s = "(" + ",".join(["'"+fetched_after_tracks[i]+"'" for i in Q]) + ")"
#         phis = dict([[a[0], float(a[1])] for a in c.execute(f"SELECT track_uri1, COUNT(*) FROM seq2_simple WHERE track_uri1 in {Q_s} AND track_uri2 = '{t}' GROUP BY track_uri1 ORDER BY track_uri1;").fetchall()])
#     # else:
#     #     Q_s2 = "(" + ",".join(["'"+fetched_prev_tracks[i]+"'" for i in Q]) + ")"
#     #     p2 = np.transpose([[a[0], int(a[1])] for a in c.execute(f"SELECT track_uri1, COUNT(*) FROM seq2_simple WHERE track_uri1 in {Q_s} AND track_uri2 = '{t}' GROUP BY track_uri1 ORDER BY track_uri1;").fetchall()])
    
#     #     Q_s3 = "(" + ",".join(["'"+fetched_after_tracks[i]+"'" for i in Q]) + ")"
#     #     p3 = np.transpose([[b[0], int(b[1])] for b in c.execute(f"SELECT track_uri2, COUNT(*) FROM seq2_simple WHERE track_uri1 = '{t}' AND track_uri2 in {Q_s} GROUP BY track_uri2 ORDER BY track_uri2;").fetchall()])
    
#     #     print(p2[:50])
#     #     print(p3[:50])
#     #     print(len(p2[0]), len(p3[0]))
#     #     phis = np.sum([p2[1], p3[1]], axis=0)

#     print("got to here 1")
#     if False: # future variability in pattern size, adding weighting based on length
#         alpha = 0.5
#         theta = 2
#         lq = len(q)
    
#     beta = 0.5 # popularity weighting

#     Q_s_filtered = "(" + ",".join(["'"+i+"'" for i in phis.keys()]) + ")"
#     counts = dict(c.execute(f"SELECT track_uri, COUNT(*) FROM playlist_tracks WHERE track_uri in {Q_s_filtered} GROUP BY track_uri ORDER BY track_uri;").fetchall())
#     psis = np.power(list(counts.values()), beta)
#     # print("phis[1]:", len(np.array(phis[1])), np.array(phis[1]))
#     # print("psis:", len(np.array(psis)), np.array(psis))
#     # rels = np.divide(np.array(phis[1]), np.array(psis))
#     rels = np.divide(psis, [phis[i] for i in phis.keys()])

#     print("got to here 2")
#     print("rels:", str(rels))

#     rel_d = {}
#     if setting == 0:
#         for pid in Q:
#             if fetched_prev_tracks[pid] in phis.keys():
#                 if rel_d[pid] is not None:
#                     rel_d[pid] += rels[phis.keys().index(fetched_prev_tracks[pid])]
#                 else:
#                     rel_d[pid] = rels[phis.keys().index(fetched_prev_tracks[pid])]
#     elif setting == 1:
#         for pid in Q:
#             if fetched_after_tracks[pid] in phis.keys():
#                 if rel_d[pid] is not None:
#                     rel_d[pid] += rels[phis.keys().index(fetched_after_tracks[pid])]
#                 else:
#                     rel_d[pid] = rels[phis.keys().index(fetched_after_tracks[pid])]
#     # else:
#     #     for pid in Q:
#     #         if fetched_after_tracks[pid] in p3[0]:
#     #             if rel_d[pid] is not None:
#     #                 rel_d[pid] += rels[p2[0].index(fetched_after_tracks[pid])]
#     #             else:
#     #                 rel_d[pid] = rels[p2[0].index(fetched_after_tracks[pid])]
#     #         elif fetched_prev_tracks[pid] in p2[0]:
#     #             if rel_d[pid] is not None:
#     #                 rel_d[pid] += rels[p2[0].index(fetched_prev_tracks[pid])]
#     #             else:
#     #                 rel_d[pid] = rels[p2[0].index(fetched_prev_tracks[pid])]
                
#     return rel_d

In [15]:
# batch_rel(0)

In [16]:
# def merge_with(d1, d2, fn=lambda x, y: x + y):
#     res = d1.copy() # "= dict(d1)" for lists of tuples
#     for key, val in d2.iteritems(): # ".. in d2" for lists of tuples
#         try:
#             res[key] = fn(res[key], val)
#         except KeyError:
#             res[key] = val
#     return res

In [17]:
# intersection = fetched_prev_tracks.keys() & fetched_after_tracks.keys()
# with Pool(len(set(fetched_prev_tracks.keys()).union(set(fetched_after_tracks.keys())))) as p:
with Pool(16) as p:
    t2, t3 = p.map(batch_rel, [0, 1])
    # t2 = p.apply_async(batch_rel, [fetched_after_tracks.keys(), 0])
    # t3 = p.apply_async(batch_rel, [fetched_prev_tracks.keys(), 1])
    # t2 = p.apply_async(batch_rel, [fetched_prev_tracks.keys() - fetched_after_tracks.keys(), 0])
    # t3 = p.apply_async(batch_rel, [fetched_after_tracks.keys() - fetched_prev_tracks.keys(), 1])
    # t1 = p.apply_async(batch_rel, [intersection, 2])
    
    p.close()
    p.join()


got to here altgot to here

LEN Q[0]: 30
got to here 1got to here 1

LEN PHI:LEN PHI:  2935

got to here 2
got to here 2
rels: rels:[  2.23606798   1.41421356   2.82842712   2.           2.
   2.82842712   6.           3.           7.87400787  10.19803903
   1.73205081   2.64575131   8.54400375   3.87298335   3.31662479
   1.           3.16227766   1.41421356   9.64365076  28.70540019
   2.64575131   6.55743852   8.06225775  76.68767828   1.
 114.52947219   2.           2.44948974   3.16227766] 
[ 21.04756518  85.72047597   1.           1.           7.81024968
   4.           2.23606798   2.23606798  10.19803903   5.65685425
   1.           2.           7.14142843   2.44948974  13.
   1.58113883   8.60232527  12.88409873   2.           9.16515139
   8.48528137   6.40312424   1.         125.67418191  28.19574436
   6.           5.19615242   1.           1.73205081   1.41421356
   9.94987437   3.16227766   7.93725393   1.73205081   5.        ]rel_d:
 rel_d:{'14153': 3.872983346207417, '1

In [18]:
from collections import Counter
# total_rels = merge_with(t2.get(), t3.get())

total_rels = Counter(t2) + Counter(t3)

In [19]:
tr_tuples = list(total_rels.items())
tr_tuples

[('14153', 13.822857717273617),
 ('15302', 9.937253933193773),
 ('78065', 8.024579547452822),
 ('125047', 7.0),
 ('211658', 134.21818566029958),
 ('220174', 9.643650760992955),
 ('225772', 14.0),
 ('249166', 120.93259642690903),
 ('254350', 114.42587615576201),
 ('295768', 2.8284271247461903),
 ('301181', 12.06225774829855),
 ('313249', 4.645751311064591),
 ('326585', 11.198039027185569),
 ('350627', 4.377802118633468),
 ('401528', 3.449489742783178),
 ('411925', 23.283633157348977),
 ('414928', 11.80190616459397),
 ('438160', 4.576491222541474),
 ('493985', 79.84995594031975),
 ('514308', 3.0),
 ('580523', 8.289489331870877),
 ('589145', 20.758106600736937),
 ('592652', 13.810249675906654),
 ('608169', 11.16515138991168),
 ('653517', 12.434107004685359),
 ('663218', 5.23606797749979),
 ('773398', 31.358022019911747),
 ('791373', 4.82842712474619),
 ('815112', 8.873479236111727),
 ('979011', 4.162277660168379),
 ('261469', 5.656854249492381),
 ('263404', 1.0),
 ('294138', 1.0),
 ('3009

In [20]:
tr_tuples_sorted = sorted(tr_tuples, key = lambda relevance: relevance[1], reverse=True)
tr_tuples_sorted = np.transpose(tr_tuples_sorted) # [:50] maybe

In [21]:
def slim_var(song_features): # todo: fine-tuning deltas and gammas
    delta_diffs = [0.25, 1, 0.25, 0.25, 0.25]
    gammas = [3, 3, 3, 3, 3]
    feature_list = np.array(song_features).transpose()
    total_attrs = [var_attr(feature_list[i], gammas[i], delta_diffs[i]) for i in range(len(gammas))]
    total = 1
    for attr in total_attrs:
        total *= attr

    return total

In [22]:
s = ",".join(["'"+t+"'" for t in tr_tuples_sorted[0]])

song_features = conn.execute(f"SELECT pt.pid, tf.acousticness, tf.instrumentalness, tf.speechiness, tf.energy, tf.valence FROM track_features as tf JOIN playlist_tracks as pt WHERE tf.track_uri = pt.track_uri AND pt.pid in ({s}) ORDER BY pt.pid ASC, pt.pindex ASC;").fetchall()

playlist_features = {}
for song in song_features:
    if song[0] not in playlist_features.keys():
        playlist_features[int(song[0])] = [(song[1],song[2],song[3],song[4],song[5])]
    else:
        playlist_features[int(song[0])].append((song[1],song[2],song[3],song[4],song[5]))


In [23]:
with Pool(16) as p:
    variances = p.map(slim_var,[playlist_features[int(pid)] for pid in tr_tuples_sorted[0]])
    p.close()
    p.join()

rhos = np.multiply(np.array(variances).astype(float), tr_tuples_sorted[1].astype(float))


totals = [(tr_tuples_sorted[0][i], rhos[i]) for i in range(len(rhos))]
containing_sorted = sorted(totals, key = lambda prevalence: prevalence[1], reverse=True)

best_k_playlists = [p[0] for p in containing_sorted[:k]] # list of k sorted playlist pids
best_k_playlists

['663218',
 '592652',
 '261469',
 '300974',
 '294138',
 '220174',
 '301181',
 '78065',
 '815112',
 '263404',
 '580523',
 '254350',
 '14153',
 '326585',
 '411925',
 '15302',
 '414928',
 '791373',
 '225772',
 '401528']

In [24]:

best_k_playlists_string = ",".join([f"'{b}'" for b in best_k_playlists])
candidate_songs = [s[0] for s in cur.execute(f"SELECT DISTINCT track_uri FROM playlist_tracks WHERE pid in ({best_k_playlists_string});").fetchall()]
current_playlist = [seed]
candidate_playlists = [current_playlist]
candidate_songs

['spotify:track:2vrIhI9rgsAmJtNlBZhca9',
 'spotify:track:4Pn8gjtVAsG9U1lmD8cmOm',
 'spotify:track:4KMecvJ7Xaps0RUqaQzfN3',
 'spotify:track:3LqLpgyNp9s8puIUXvvMPx',
 'spotify:track:4wMhPDmx9Ov4VKS2NNN28B',
 'spotify:track:0F8psXXh2oxE5BGdkKaoIs',
 'spotify:track:04tI5D7yVvxixXoT0QYrdj',
 'spotify:track:33w7LgNJIuhs5dhSYZ0do1',
 'spotify:track:5s0AxcVJJD2YYPstMb61ro',
 'spotify:track:38kcwx5znepLWCIBID0Bwm',
 'spotify:track:5sqkarfxe7UejHTlCtHCLS',
 'spotify:track:1Wh65eWI9v6yg44NJhy7BU',
 'spotify:track:1vYVybuvtgTMXQk7iEPJ7y',
 'spotify:track:4xm8SvpjyGP2Bs42EO4yyL',
 'spotify:track:0IWtO0I9UWvYrQYNHMJstU',
 'spotify:track:3SfnsExYXLlbW9NByxb4L2',
 'spotify:track:6xG2ZGudUgtV235xvDlSEt',
 'spotify:track:1hi0P1Y2he1p09xnUccuAN',
 'spotify:track:6VBQ4DF4fwPJNA96RbMQWT',
 'spotify:track:0f34MnV51uwA1jhPQ2XnDe',
 'spotify:track:3hsyn73e03cL3JCnlRIQ7K',
 'spotify:track:3QjIdDDKoZRyfWDKZC4Ayb',
 'spotify:track:0l6uRRIDPNw3OsmqCYkmvi',
 'spotify:track:4n5f6vydWvNsCcQH6p1Bmw',
 'spotify:track:

In [25]:
def check_pair(x, y): # check whether sequence <x, y> occurs in any of the k retrieved playlists and count them/return existence, AND p1.pindex + 1 = p2.pindex
    # occurences = 0
    # import time
    # t0 = time.time() 135 seconds???
    return cur.execute(f"SELECT COUNT(p1.pid) FROM playlist_tracks as p1 JOIN playlist_tracks as p2 WHERE p1.track_uri = '{x}' AND p2.track_uri = '{y}' AND p1.pid = p2.pid AND p1.pid in ({best_k_playlists_string}) AND p1.pindex + 1 = p2.pindex;").fetchone()[0]
    # t1 = time.time()
    # print(str(t1-t0))
    # for playlist in containing_playlists:
    #     if playlist[1] == playlist[2] - 1:
    #         return 1
            # occurences += 1
    # return occurences

In [26]:
def check_batch(x, y, pre):
    '''
    Check whether sequence <x, y> occurs in any of the k retrieved playlists and count them where either x is the song in the playlist and y is a list of candidates or the other way round. Returns list of candidates, occurence count for each is almost always at most 1 so was removed
    '''
    if pre:
        return cur.execute(f"SELECT p1.track_uri FROM playlist_tracks as p1 JOIN playlist_tracks as p2 WHERE p1.track_uri in ({x}) AND p2.track_uri = '{y}' AND p1.pid = p2.pid AND p1.pid in ({best_k_playlists_string}) AND p2.pindex - 1 = p1.pindex GROUP BY p1.track_uri;").fetchall()
    else:
        return cur.execute(f"SELECT p1.track_uri FROM playlist_tracks as p1 JOIN playlist_tracks as p2 WHERE p1.track_uri in ({x}) AND p2.track_uri = '{y}' AND p1.pid = p2.pid AND p1.pid in ({best_k_playlists_string}) AND p2.pindex + 1 = p1.pindex GROUP BY p1.track_uri;").fetchall()

In [27]:
def h_old(T_prime, seq, u):
    r = rel(seq, u) # relevance of pattern seq to added song u
    return r * var(T_prime)

    # todo: look-ahead factor L

In [28]:
def slim_rel(phi, popularity):
    
    if False: # future variability in pattern size, adding weighting based on length
        alpha = 0.5
        theta = 2
        lq = len(q)
    
    beta = 0.5 # popularity weighting
    psi = 0

    psi = pow(popularity, beta)
    rel = phi / psi

    return rel

In [29]:
def h(phi, pop, song_features):
    return slim_rel(phi, pop) * slim_var(song_features)

    # todo: look-ahead factor L

In [30]:
N = 10

# blacklist = [seed]
candidate_songs_str = ",".join(["'"+c+"'" for c in candidate_songs])

song_features = conn.execute(f"SELECT pt.track_uri, tf.acousticness, tf.instrumentalness, tf.speechiness, tf.energy, tf.valence FROM track_features as tf JOIN playlist_tracks as pt WHERE tf.track_uri = pt.track_uri AND pt.track_uri in ({candidate_songs_str},'{seed}') ORDER BY pt.pid ASC, pt.pindex ASC;").fetchall()
song_f = {}
for song in song_features:
    song_f[song[0]] = song[1:]

song_popularity = conn.execute(f"SELECT track_uri, COUNT(*) FROM playlist_tracks WHERE track_uri in ({candidate_songs_str}) GROUP BY track_uri;").fetchall()
song_p = {}
for song in song_popularity:
    song_p[song[0]] = int(song[1])

In [31]:

while len(current_playlist) < N:
    t = current_playlist[0]
    T = current_playlist[-1]
    successors_of_current_playlist = []
    
    candidate_songs_str = ",".join(["'"+c+"'" for c in candidate_songs if c not in current_playlist])

    pres = check_batch(candidate_songs_str, t, True)
    rel_buffer_pres = []
    if pres:
        for pre in pres:
            successor_playlist = [pre[0]] + current_playlist
            successors_of_current_playlist.append([successor_playlist, pre[0], True])
            rel_buffer_pres.append(pre[0])

    posts = check_batch(candidate_songs_str, T, False)
    rel_buffer_posts = []
    if posts:
        for post in posts:
            successor_playlist = current_playlist + [post[0]]
            successors_of_current_playlist.append([successor_playlist, post[0], False])
            rel_buffer_posts.append(post[0])

    if len(successors_of_current_playlist) == 0:
        # discard current_playlist
        print("failed to create playlist")
    else: # pop new current_playlist from candidate_playlists
    	# sort successors_of_current_playlist using some measure of quality 

        # successors_of_current_playlist = np.array(successors_of_current_playlist) parallel compute??

        # for x in range(len(successors_of_current_playlist)): # todo: more efficient var check?
        #     successors_of_current_playlist[x][3] = h(successors_of_current_playlist[x][1], successors_of_current_playlist[x][2], x)

        phis_pres = batch_phi(True, rel_buffer_pres, current_playlist[0])
        phis_posts = batch_phi(False, rel_buffer_posts, current_playlist[-1])

        succ_data = []
        for i, succ in enumerate(successors_of_current_playlist):
            if succ[2]:
                phi = phis_pres[succ[1]]
            else:
                phi = phis_posts[succ[1]]
            succ_data.append((phi, song_p[succ[1]], [song_f[x] for x in succ[0]]))

        with Pool(32) as p:
            ratings = p.starmap(h, succ_data)
            p.close()
            p.join()
            
        for i in range(len(successors_of_current_playlist)):
            successors_of_current_playlist[i].append(ratings[i])

        successors_of_current_playlist = sorted(successors_of_current_playlist, key=lambda x: x[3], reverse=True)
        print(successors_of_current_playlist)
        current_playlist = successors_of_current_playlist[0][0]
        saved_playlists = successors_of_current_playlist[1::]
        # insert the saved_playlists into candidate_playlists
        candidate_playlists += saved_playlists
        # In fact, they should be inserted in such a way as to keep candidate_playlists 
        # ordered also by their quality, but ignore this for now

# If we reach here, we exited the loop, i.e. we have a playlist of length L
print(current_playlist)


[[['spotify:track:0d9A17LiAJra5hI1ocJuHV', 'spotify:track:3mScGCzxiXA9OaHdBeuk7O'], 'spotify:track:0d9A17LiAJra5hI1ocJuHV', True, 1.0], [['spotify:track:23cTq6ZjYo0wDRNrdkaUOC', 'spotify:track:3mScGCzxiXA9OaHdBeuk7O'], 'spotify:track:23cTq6ZjYo0wDRNrdkaUOC', True, 1.0], [['spotify:track:3mScGCzxiXA9OaHdBeuk7O', 'spotify:track:3rb5K4Cw0GsLsgjcSYAP9K'], 'spotify:track:3rb5K4Cw0GsLsgjcSYAP9K', False, 1.0], [['spotify:track:3mScGCzxiXA9OaHdBeuk7O', 'spotify:track:1tPUW9zWR8VwkJfP9p3Fl4'], 'spotify:track:1tPUW9zWR8VwkJfP9p3Fl4', False, 0.5773502691896258], [['spotify:track:5AngW69uAboljtCBHZ6AsT', 'spotify:track:3mScGCzxiXA9OaHdBeuk7O'], 'spotify:track:5AngW69uAboljtCBHZ6AsT', True, 0.5], [['spotify:track:1hP8CqEdjSzUHQAP3aXsKZ', 'spotify:track:3mScGCzxiXA9OaHdBeuk7O'], 'spotify:track:1hP8CqEdjSzUHQAP3aXsKZ', True, 0.4472135954999579], [['spotify:track:3mScGCzxiXA9OaHdBeuk7O', 'spotify:track:03Mlqofvb0zaPNJeqSFMHB'], 'spotify:track:03Mlqofvb0zaPNJeqSFMHB', False, 0.4472135954999579], [['spo

In [34]:
",".join(current_playlist)

'spotify:track:6pAYtrkxFGefYqYPBfvzPy,spotify:track:4Cv5PfRm2Qefuaxapdk9XQ,spotify:track:7qrqQjdaJalneb5T62e21z,spotify:track:2kjLcWz7liVad1zg35iL7D,spotify:track:0d9A17LiAJra5hI1ocJuHV,spotify:track:3mScGCzxiXA9OaHdBeuk7O,spotify:track:3rb5K4Cw0GsLsgjcSYAP9K,spotify:track:0KKkJNfGyhkQ5aFogxQAPU,spotify:track:5meZvaHT6pQx4AZP1f8lN3,spotify:track:4KMecvJ7Xaps0RUqaQzfN3'

In [33]:
current_playlist_str = ",".join([song[14:] for song in current_playlist])
current_playlist_str

'6pAYtrkxFGefYqYPBfvzPy,4Cv5PfRm2Qefuaxapdk9XQ,7qrqQjdaJalneb5T62e21z,2kjLcWz7liVad1zg35iL7D,0d9A17LiAJra5hI1ocJuHV,3mScGCzxiXA9OaHdBeuk7O,3rb5K4Cw0GsLsgjcSYAP9K,0KKkJNfGyhkQ5aFogxQAPU,5meZvaHT6pQx4AZP1f8lN3,4KMecvJ7Xaps0RUqaQzfN3'

Toto - Hold the Line: `2771LMNxwf62FTAdpJMQfM,2dCcDpm6O6ocomusl5ao6p,5o4yGlG0PfeVUa6ClIyOxq,11dxtPJKR4E0wlSr0A0t47,37BTh5g05cxBIRYMbw8g2T,4aVuWgvD0X63hcOCnZtNFA,2374M0fQpWi3dLnB54qaLX,3OZ40egQbNWeTe0BnR2QKa,3T6R0ppqCnUW6iJhtUYhO3,18sytW2s53Of6NVudQyUlH`

Britney Spears - Toxic: `2mfUa8bLs2s5N4VaqJZ4lZ,53mrVsi49rLHIaKBiSvElG,0NiXXAI876aGImAd6rTj8w,4fixebDZAVToLbUCuEloa2,0ofbQMrRDsUaVKq2mGLEAb,1louJpMmzEicAn7lzDalPW,2ekn2ttSfGqwhhate0LSR0,3KliPMvk1EvFZu9cvkj8p1,6I9VzXrHxO9rA9A5euc8Ak,2TTYIwTM2iLC1YOyHuhRMt`

Ken Ashcorp - On The Rocks: `3qI94hINNNeb4S7xQi18lS,6XEInZxmXYXTtbu57F8mZi,5pyw9ufwba90TEl8HMzdDI,1z9t09cCzfOz9R8UYjirt8,4SenxwCmSCIXfklUvmXyNc,32dlXisrLwN3en5L6gfjsY,7o7XA7UMKl3Dm1d3sEBwwM,4NCuItDZQ5kP5gL3UTG0zs,4SYZZNgqbsyH3FveJFrYUA,7mwWiWQq4cVICZKIcSJ08L`

Kancolle - Miiro: `6frt4YwBI8tOZ3dvL932VC,6xRzEQ4rQnQmfXJWisT06K,6JoRyy8rJI7yweRlJfQmyW,0vHBV9HNgb0GjAF9UnhPf6,6LlvwXe8c1edvjqT68mwJz,754jwwYVyaKQNavm2YMxNL,0kTZWRzfmqDN4boykkjUXH,3LOg3UdLGzOMrYaNYxrIfR,4LsGVuHpNGvMGiHwMTqbc8,0GNl4scgMHaORIZ8WhIvHu`